## Using MoSH to fit body models to labeled mocaps

We base this off of the two SOMA tutorials: [Run SOMA on SOMA dataset](https://github.com/nghorbani/soma/blob/main/src/tutorials/run_soma_on_soma_dataset.ipynb) and [Solve Already Labeled MoCaps with Mosh++](https://github.com/nghorbani/soma/blob/main/src/tutorials/solve_labeled_mocap.ipynb).

Reading the [definitions](https://github.com/nghorbani/soma/tree/main/src/tutorials#definitions) of terminology is recommended.

MoSH++ takes in labeled motion capture data and fits an SMPL body model to it. As a note, the original MoSH paper converted the motion capture data to a different body model; we use MoSH/MoSH++ interchangeably, with both referring to the MoSH++.

A *labeled* motion capture is simply one that has markers assigned to the point cloud points for each frame. The exact names of the markers is not relevant, just their stability (Is the marker mapped to the same point consistently for each frame)  (TODO: verify this).

From the [MoSH paper](https://files.is.tue.mpg.de/black/papers/MoSh.pdf):

> Marker placement on the human body varies across subjects and sessions, consequently we do not assume that the exact marker placement is known. Instead, a key contribution of MoSh is that it solves for the observed marker locations relative to the 3D body model.

The MoSH algorithm works in two steps:

1. Solving for Beta Parameters. A single frame from the sequence is picked, and the SMPL body model is fit to the frame. SMPL models, in their simplest form, have 82 parameters per frame: 23 joints, 3 parameters to represent each joint in 3d space with axis-angle rotation, 3 more parameters to represent the root of the body (global translation), and 10 Beta parameters to characterize the variation of body height, body proportion, and weight. Once these Beta Parameters are solved for, they are fixed, and are used to fit the rest of the frames in the sequence. This process can take 20-30minutes per subject.
2. With Betas fixed, MoSH++ fits the SMPL body model frame by frame for the sequence. This process takes ~2 seconds per frame.

TODO: cite the paper that has the approximate times for each step.

TODO: bring in info about how MoSH uses temporal smoothing for its frame-by-frame fitting.
